In [1]:
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.optimize import minimize
stops = set(stopwords.words("english"))
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import multiprocessing
import difflib

/home/takaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
features_for_input = [
    'q1_lemmatized',
    'q2_lemmatized'
]
def read_data(t):
    data = pd.read_csv('../input/{}.csv'.format(t))
    for feature in features_for_input:
        data = pd.merge(data, pd.read_pickle('../feature/{}_{}.pkl'.format(t, feature)))
    data.fillna(0.0)
    return data

train = read_data('train')#[:1000]
#test = read_data('test')[:1000]

In [14]:
def s2v(row):
    w1 = [w for w in row['q1_lemmatized'] if w not in stops and w in model]
    w2 = [w for w in row['q2_lemmatized'] if w not in stops and w in model]
    if w1 and w2:
        s1 = sum(model[w] for w in w1)
        s2 = sum(model[w] for w in w2)
        return (s1, s2)
    else:
        return ([], [])

In [5]:
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin.gz', binary=True)

2017-04-03 04:48:51,286 : INFO : loading projection weights from ../data/GoogleNews-vectors-negative300.bin.gz
2017-04-03 04:51:20,957 : INFO : loaded (3000000, 300) matrix from ../data/GoogleNews-vectors-negative300.bin.gz


In [15]:
W2V_LEN = 300
import gc
def create_id_to_s2vec(data):
    q1_word2vec_freatures = ['q1_word2vec{:03d}'.format(i) for i in range(W2V_LEN)]
    q2_word2vec_freatures = ['q2_word2vec{:03d}'.format(i) for i in range(W2V_LEN)]
    
    for index, row in data.iterrows():
        gc.collect()
        if index % 10000 == 0:
            print(index)

        s1, s2 = s2v(row)

        if len(s1) > 0:
            for i in range(W2V_LEN):
                data.set_value(index, q1_word2vec_freatures[i], s1[i])
            for i in range(W2V_LEN):
                data.set_value(index, q2_word2vec_freatures[i], s2[i])

In [16]:
add_word2vec(train)
#add_word2vec(test)

0


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
train[['id'] + features].to_pickle('../feature/train_word2vec')
#test[['test_id'] + features].to_pickle('../feature/test_word2vec')